# Recurrent Neural Networks

Traditional artificial neural networks obtain "long-term memory" by learning the adequate weights. The idea behind recurrent neural networks is to model "short-term memory". By connecting the hidden layer of the last observation to the hidden layer of the current observation, we allow the network to include knowledge of previous layers.

A generic RNN, with input $u_t$ and state $x_t$ for time step $t$ is given by:

$$x_t=F(x_{t-1}, u_t, \theta)$$

<img src='resources/rnn.png' width=800>

A task displays long-term dependencies if prediction of the desired output at time $t$ depends on input presented at an earlier time $\tau < t$. Bengio *et al*. (1994) proposed that a dynamical system that can learn to store relevant state information requires the following: 

1. Ability to store information for an arbitrary duration (information latching).
2. Resistance to noise.
3. Ability to train parameters in reasonable time.

Traditional backpropagation in RNN is not sufficiently powerful to discover contingencies spanning long temporal intervals, resulting in parameters settling in sub-optimal solutions focused on short-time dependencies, but not long-term dependencies. This is mainly due to the vanishing and exploding gradient problems, in which the gradient either exponentially loses or gains magnitude as it travels back through the network.

There are several solutions to this problem, including:

* Long Short-Term Memory Networks
* Echo-state Networks
* Hessian-Free optimization with structural damping
* Gradient clipping
* Vanishing gradient regularization

## Types of RNN

* One-to-Many - image captioning
* Many-to-One - sentiment analysis
* Many-to-Many - neural machine translation, video captioning

<img src='resources/types.png' width=600>

## References

* Bengio, Y., Simard, P., and Frasconi, P. (1994). [Learning Long-Term Dependencies with Gradient Descent is Difficult](http://ai.dinfo.unifi.it/paolo//ps/tnn-94-gradient.pdf).
* Pascanu, R., Mikolov, T., and Bengio, Y., (2013). [On the difficulty of training recurrent neural networks](http://proceedings.mlr.press/v28/pascanu13.pdf).
* [Long Short-Term Memory](http://www.bioinf.jku.at/publications/older/2604.pdf)
* [Understanding LSTM Networks](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)
* [Understanding LSTM and its diagrams](https://medium.com/mlreview/understanding-lstm-and-its-diagrams-37e2f46f1714)
* [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)
* [Visualizing and Understanding Recurrent Networks](https://arxiv.org/pdf/1506.02078.pdf)
* [LSTM: A Search Space Odyssey](https://arxiv.org/pdf/1503.04069.pdf)

# Long Short-Term Memory Networks

LSTMs are a type of recurrent neural network module designed to avoid the vanishing and exploding gradients problem.

<img src='resources/lstm.png'>

<img src='resources/lstm-components.png'>

## Cell State

<img src='resources/lstm-cell-state.png'>

The cell state is the core idea behind LSTMs. It is like a conveyor belt, running straight down the entire network, with only some linear interactions, making it easy for information to flow along it unchanged. 

The LSTM has the ability to remove or add information to the cell state, regulated by gates. Gates are a way to optionally let information through. They are composed out of a sigmoid activation function and a pointwise operation.

## Forget Gate Layer

The forget gate layer decides which information to throw away from the cell state. This decision is made by outputting a number between $0$ and $1$ for each number in the cell state $C_{t-1}$.

For example, the network might want to remember the gender of the present subject, so that the correct pronouns can be used. When a new subject is seen, we want to forget the gender of the old subject.

<img src='resources/lstm-forget-gate.png'>

$$f_t=\sigma(W_f(h_{t-1}\oplus x_t) + b_f)$$

## Input Gate Layer

The next step is to decide what information to store in the cell state. First, a sigmoid layer decides which values we'll update. Next, a $\tanh$ layer creates a vector of new candidate values, $\tilde{C}_t$, that could be added to the cell state. Finally, these two are combined to create an update to the state.

<img src='resources/lstm-input-gate.png'>

$$ i_t = \sigma(W_i(h_{t-1}\oplus x_t) + b_i) $$

$$ \tilde{C}_t = \tanh(W_C(h_{t-1}\odot x_t) + b_C) $$

Now, we update the cell state by forgetting the things we decided to forget, and then adding the new candidate values, scaled by how much we decided to update each state value.

<img src='resources/lstm-cell-update.png'>

$$ C_t = f_tC_{t-1} + i_t\tilde{C}_t $$

For example, this is the moment where the cell state forgets the previous subject's gender and remembers the current subject's gender.

## Output Gate Layer

Finally, we need to decide what we're going to output. This will be based on our cell state, but will be a filtered version. First, we run a sigmoid layer which decides which part of the cell state we're going to output. Then, we put the cell state through a $\tanh$ (to push the values between $-1$ and $1$), and multiply these two together so that we only output the parts we decided to.

<img src='resources/lstm-output-gate.png'>

$$ o_t = \sigma(W_O(h_{t-1}\oplus x_t) + b_O) $$

$$ h_t = o_t\tanh(C_t) $$

Here, the network might output information relevant to the next layers. For example: whether the subject is singular or plural.

# Variants on Long Short-Term Memory Networks

## LSTM with peephole connections

Allow gate layers to look at the cell state.

<img src='resources/lstm-peepholes.png'>

$$ f_t = \sigma(W_f(C_{t-1}\oplus h_{t-1}\oplus x_t) + b_f) $$

$$ i_t = \sigma(W_i(C_{t-1}\oplus h_{t-1}\oplus x_t) + b_i) $$

$$ o_t = \sigma(W_o(C_{t}\oplus h_{t-1}\oplus x_t) + b_o) $$

## LSTM with coupled forget and input gates

Jointly decide which information to forget and add. Only forget when we're going to input something in its place, and only input new values to the state when we forget something older.

<img src='resources/lstm-coupled.png'>

$$ C_t = f_tC_{t-1} + (1-f_t)\tilde{C}_t $$

## Gated Recurrent Units

GRUs combine the forget and input gates into a single "update gate". Also merges the cell state and hidden state, and makes some other changes. The resulting model is simpler than standard LSTM models and has been growing increasingly popular.

<img src='resources/gru.png'>

$$ z_t = \sigma(W_z(h_{t-1}\oplus x_t)) $$

$$ r_t = \sigma(W_r(h_{t-1}\oplus x_t)) $$

$$ \tilde{h}_t = \tanh(W(r_th_{t-1}\oplus x_t)) $$

$$ h_t = (1-z_t)h_{t-1} + z_t\tilde{h}_t $$

## Other extensions

* Depth-Gated RNNs https://arxiv.org/pdf/1508.03790v2.pdf
* Clockwork RNNs https://arxiv.org/pdf/1402.3511v1.pdf
* Attention mechanisms https://arxiv.org/pdf/1502.03044v2.pdf
* Grid LSTMs https://arxiv.org/pdf/1507.01526v1.pdf
* RNNs in generative models https://arxiv.org/pdf/1502.04623.pdf, https://arxiv.org/pdf/1506.02216v3.pdf, https://arxiv.org/pdf/1411.7610v3.pdf
* [Greff, et al. (2015)](http://proceedings.mlr.press/v37/jozefowicz15.pdf) compare several popular variants, finding that they're all about the same. [Jozefowicz, et al. (2015)](http://proceedings.mlr.press/v37/jozefowicz15.pdf) tested thousands of RNN architectures, finding some that worked better than LSTMs on certain tasks.